In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

import re
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
train = pd.read_csv('../input/train.csv')
# Any results you write to the current directory are saved as output.

In [ ]:
y = train['SalePrice']
x1 = train.drop(['SalePrice'],axis = 1)

x1 = x1.loc[:,'MSSubClass':'SaleCondition']
ntrain = len(x1)


In [ ]:
sns.distplot(y,fit = norm)
# a liltle bit skew
(mu,sigma) = norm.fit(y)
fig = plt.figure()
res = stats.probplot(y,plot = plt)
plt.show()



In [ ]:
y = np.log1p(y)
sns.distplot(y,fit = norm)
norm.fit(y)

In [ ]:
test1 = pd.read_csv('../input/test.csv')
test_ID = test1['Id']

In [ ]:
test1 = pd.read_csv('../input/test.csv')
#test = pd.get_dummies(test)
#test = test.fillna(test.mean())
#test.shape

test1.head()
ntest = len(test1)
all_data = pd.concat((x1.loc[:,'MSSubClass':'SaleCondition'],test1.loc[:,'MSSubClass':'SaleCondition']))

In [ ]:

all_data['MiscFeature'] = all_data['MiscFeature'].fillna('None')
all_data['Fence'] = all_data['Fence'].fillna('None')
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
all_data["LotFrontage"] = all_data['LotFrontage'].fillna(0)
all_data['Alley'] = all_data["Alley"].fillna('None')
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
#all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

#percen = all_data.isnull().sum() / len(all_data)
#percen.sort_values(ascending= False)

In [ ]:
all_data['PoolQC'] = all_data['PoolQC'].fillna('None')
#percen = all_data.isnull().sum() / len(all_data)
#percen.sort_values(ascending= False)
all_data.dtypes[all_data.dtypes != "object"].index

In [ ]:
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
numeric_features = all_data.dtypes[all_data.dtypes != "object"].index
all_data[numeric_features].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :all_data[numeric_features].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
})

skewness.head(5)


In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    all_data[feat] = boxcox1p(all_data[feat], lam)


In [ ]:
all_data = pd.get_dummies(all_data)
all_data.shape
train_f = all_data[:ntrain]
test_f = all_data[ntrain:]
train_f.isnull().sum()

In [ ]:
from sklearn.model_selection import KFold

n_folds = 5
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train_f, y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
#basic model in sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LassoCV, Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
Lasso = make_pipeline(RobustScaler(),LassoCV(alphas = [1, 0.1, 0.001, 0.0005], random_state = 1))
Enet = make_pipeline(RobustScaler(), ElasticNet(alpha = 0.0005, l1_ratio = 0.9,random_state = 3))
Kridge = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
model_xgb = XGBRegressor(base_score = 0.5,colsample_bylevel = 0.7, colsample_bytree =0.75,
                    max_depth = 5,
                    min_child_weight = 6,
                   gamma = 0,
                   learning_rate = 0.1,
                   max_delta_step = 0,
                   missing = None,
                   n_estimator = 200,
                   nthread = -1,
                   objective = 'reg:linear',
                   reg_alpha = 100,
                   reg_lambda = 0.8,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
Lgbm = LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
score =rmsle_cv(Lasso)
print ("Lasso:{}".format(score))
score =rmsle_cv(Enet)
print ("Enet:{}".format(score))
score =rmsle_cv(Kridge)
print ("Kridge:{}".format(score))
#score =rmsle_cv(xgb)
print ("xgb:{}".format(score))
#score =rmsle_cv(GBoost)
print ("Gboost:{}".format(score))
#score =rmsle_cv(Lgbm)
print ("Lgbm:{}".format(score))

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [ ]:
class AveragingModels(BaseEstimator,RegressorMixin, TransformerMixin):
    def __init__(self,models):
        self.models = models
    def fit(self,x,y):
        self.models_ = [clone(x) for x in self.models]
        
        for model in self.models_:
            model.fit(x,y)
        return self

    def predict(self, x):
        predictions = np.column_stack([
            model.predict(x) for model in self.models_
        ])
        return np.mean(predictions,axis = 1)
    

In [ ]:
averaged_models = AveragingModels(models = (Enet, GBoost, Kridge, Lasso))
score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (Enet, GBoost, Kridge),meta_model = Lasso)
score = rmsle_cv(stacked_averaged_models)


In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
stacked_averaged_models.fit(train_f.values, y)
stacked_train_pred = stacked_averaged_models.predict(train_f)
stacked_pred = np.expm1(stacked_averaged_models.predict(test_f))
print(rmsle(y, stacked_train_pred))

In [ ]:
model_xgb.fit(train_f,y)
xgb_train_pred = model_xgb.predict(train_f)
xgb_pred = np.expm1(model_xgb.predict(test_f))
print(rmsle(y, xgb_train_pred))

In [ ]:
Lgbm.fit(train_f,y)
Lgmb_train_pred = Lgbm.predict(train_f)
Lgbm_pred = np.expm1(Lgbm.predict(test_f.values))
print(rmsle(y,Lgbm_train_pred))

In [ ]:
ensemble = stacked_pred*0.9 + xgb_pred*0.1
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = ensemble
sub.to_csv('submission_stacked.csv',index=False)

-----------End-----------
Below are the process to choose hyperparameter for xgb

In [ ]:

#first try on xgboost
import xgboost as xgb
from pandas import DataFrame

bst =xgb.XGBRegressor()
bst.fit(x,y,verbose = True)
preds = bst.predict(test)


In [ ]:
from sklearn import metrics
preds_train = bst.predict(x)
metrics.r2_score(y.values,preds_train)

In [ ]:
#tune the hyperparameters
from xgboost import XGBRegressor
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV
xgb3 = XGBRegressor(base_score = 0.5,colsample_bylevel = 0.7, colsample_bytree =0.75,
                    max_depth = 5,
                    min_child_weight = 6,
                   gamma = 0,
                   learning_rate = 0.1,
                   max_delta_step = 0,
                   missing = None,
                   n_estimator = 200,
                   nthread = -1,
                   objective = 'reg:linear',
                   reg_alpha = 100,
                   reg_lambda = 0.8,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1)

In [ ]:
#define the valuation
def modelfit(alg,x,y,useTrainCV = True, cv_folds = 5,early_stopping_rounds = 100):
    alg.fit(x,y,eval_metric = 'rmse')
    train_pred = alg.predict(x)
    print ("\n for this model")
    print (metrics.r2_score(y,train_pred))


In [ ]:
#parameters setting
xgb1 = XGBRegressor(base_score = 0.5,colsample_bylevel = 1, colsample_bytree = 1,
                   gamma = 0,
                   learning_rate = 0.1,
                   max_delta_step = 0,
                   max_depth = 3,
                   min_child_weight = 1,
                   missing = None,
                   n_estimator = 100,
                   nthread = -1,
                   objective = 'reg:linear',
                   reg_alpha = 0,
                   reg_lambda = 1,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1)
modelfit(xgb1,x,y)

In [ ]:
#use GridSearchCV to find better model
param_test1 = {
    'max_depth':[3,5,7,9],
    'min_child_weight':[2,4,6,8]
}

gsearch1 = GridSearchCV(estimator = XGBRegressor(base_score = 0.5,
                                                 colsample_bylevel = 1, 
                                                 colsample_bytree = 1,
                                                 gamma = 0,
                                                 learning_rate = 0.1,
                                                max_delta_step = 0,
                   missing = None,
                   n_estimator = 100,
                   nthread = -1,
                   objective = 'reg:linear',
                   reg_alpha = 0,
                   reg_lambda = 1,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1),
                   param_grid = param_test1, scoring = 'r2',cv = 5)
gsearch1.fit(x,y)
                

In [ ]:
gsearch1.grid_scores_,gsearch1.best_params_,gsearch1.best_score_

In [ ]:
param_test2 = {
    'max_depth':[6,7,8],
    'min_child_weight':[3,4,5]
}

gsearch2 = GridSearchCV(estimator = XGBRegressor(base_score = 0.5,
                                                 colsample_bylevel = 1, 
                                                 colsample_bytree = 1,
                                                 gamma = 0,
                                                 learning_rate = 0.1,
                                                max_delta_step = 0,
                   missing = None,
                   n_estimator = 100,
                   nthread = -1,
                   objective = 'reg:linear',
                   reg_alpha = 0,
                   reg_lambda = 1,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1),
                   param_grid = param_test2, scoring = 'r2',cv = 5)
gsearch2.fit(x,y)

In [ ]:
gsearch2.grid_scores_,gsearch2.best_params_,gsearch2.best_score_

In [ ]:
#parameters setting
xgb2 = XGBRegressor(base_score = 0.5,colsample_bylevel = 0.7, colsample_bytree =0.75,
                    max_depth = 8,
                    min_child_weight = 3,
                   gamma = 0,
                   learning_rate = 0.1,
                   max_delta_step = 0,
                   missing = None,
                   n_estimator = 100,
                   nthread = 1,
                   objective = 'reg:linear',
                   reg_alpha = 0,
                   reg_lambda = 1,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1)
modelfit(xgb2,x,y)

In [ ]:
#parameters setting
xgb3 = XGBRegressor(base_score = 0.5,colsample_bylevel = 0.7, colsample_bytree =0.75,
                    max_depth = 5,
                    min_child_weight = 6,
                   gamma = 0,
                   learning_rate = 0.1,
                   max_delta_step = 0,
                   missing = None,
                   n_estimator = 200,
                   nthread = -1,
                   objective = 'reg:linear',
                   reg_alpha = 100,
                   reg_lambda = 0.8,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1)
modelfit(xgb3,x,y)
xgb3.fit(x,y)

In [ ]:
xgb_preds = xgb3.predict(test)
#test1.Id.shape
#test.shape
#solution = pd.DataFrame({"id":test1.Id, "SalePrice":preds})
#solution.to_csv("ridge_sol.csv", index = False)

In [ ]:
#Version2 LR

In [ ]:


def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, x, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [ ]:
model_ridge = Ridge()

In [ ]:
#search for the best regularization parameter
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() for alpha in alphas]
import matplotlib.pyplot as plt



%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [ ]:
cv_ridge = pd.Series(cv_ridge,index = alphas)
cv_ridge.plot(title = 'Validation')
plt.xlabel('alpha')
plt.ylabel('rmse')
cv_ridge.min()

In [ ]:
#Lasso
#lasso = make_pipeline(RobustScaler(),LassoCV(alphas = [1, 0.1, 0.001, 0.0005], random_state = 1)
###model_lasso = make_pipeline(RobustScaler(),LassoCV(alphas = [1, 0.1, 0.001, 0.0005], random_state = 1).fit(x, y)
#rmse_cv(model_lasso).mean()

In [ ]:
#coef = pd.Series(model_lasso.coef_, index = x.columns)
#print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
#imp_coef = pd.concat([coef.sort_values().head(10),
#                     coef.sort_values().tail(10)])
#imp_coef

In [ ]:
#matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
#imp_coef.plot(kind = "barh")
#plt.title("Coefficients in the Lasso Model")

In [ ]:
#matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

#preds = pd.DataFrame({"preds":model_lasso.predict(x), "true":y})
#preds["residuals"] = preds["true"] - preds["preds"]
#preds.plot(x = "preds", y = "residuals",kind = "scatter")

In [ ]:
#lr_preds = model_lasso.predict(test)
#preds = 0.7*lr_preds + 0.3*xgb_preds
#pred_train = 0.3* model_lasso.predict(x) + 0.7* xgb3.predict(x)
#metrics.r2_score(y,pred_train)

In [ ]:
#solution = pd.DataFrame({"id":test1.Id, "SalePrice":preds})
#solution.to_csv("ridge_sol.csv", index = False)

In [ ]:
#V3 add neural network

#from keras.layers import Dense
#from keras.models import Sequential
#from keras.regularizers import l1
#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import train_test_split

In [ ]:
#x_nn = StandardScaler().fit_transform(x)
#x_tr,x_val,y_tr,y_val = train_test_split(x_nn,y,random_state =3 )


In [ ]:
#model = Sequential()
#model.add(Dense(1,input_dim = x.shape[1], W_regularizer=l1(0.001)))

#model.compile(loss = 'mse',optimizer = 'adam')

In [ ]:
#hist = model.fit(x_tr,y_tr,validation_data = (x_val,y_val))

In [ ]:
#nn_preds = model.predict(x_nn)

In [ ]:
#pd.Series(model.predict(x_nn)[:,0])

In [ ]:
#pred_train = 0.1* model_lasso.predict(x) + 0.8* xgb3.predict(x) + 0.1 * pd.Series(model.predict(x_nn)[:,0])
#metrics.r2_score(y,pred_train)


In [ ]:
#preds = 0.1* model_lasso.predict(test) + 0.8* xgb3.predict(test) + 0.1 * pd.Series(model.predict(test)[:,0])

In [ ]:
#solution = pd.DataFrame({"id":test1.Id, "SalePrice":preds})
#solution.to_csv("ridge_sol.csv", index = False)

In [ ]:
#The third combination worked worse than the second version